Предварительно про PyTorch:
* [Про тензоры в pytorch](https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/tensor_tutorial.ipynb)
* [Про автоматическое дифференцирование и что такое .backwards()](https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/autograd_tutorial.ipynb)
* [Очень простая нейронка на pytorch](https://colab.research.google.com/drive/1RsZvw4KBGn5U5Aj5Ak7OG2pHx6z1OSlF)

# Классификация текстов

## Fakenews

1. Мы будем работать с данными fakenews отсюда: https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv
2. Проведите препроцессинг текста. Разбейте данные на train и test для задачи классификации.
3. Векторизуйте.
4. Обучите на полученных векторах алгоритм классификации.

Мы уже видели как эта задача выполняется с помощью Word2vec. Давайте вспомним.

In [2]:
!wget https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv

--2021-12-16 10:36:27--  https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1253562 (1.2M) [text/plain]
Saving to: ‘Constraint_Train.csv’

Constraint_Train.cs 100%[===================>]   1.20M  --.-KB/s    in 0.05s   

2021-12-16 10:36:28 (23.4 MB/s) - ‘Constraint_Train.csv’ saved [1253562/1253562]



In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('Constraint_Train.csv')

In [5]:
df.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [6]:
from nltk.tokenize import word_tokenize
from tqdm import tqdm

In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
sentences = [word_tokenize(text.lower()) for text in tqdm(df.tweet)]

100%|██████████| 6420/6420 [00:02<00:00, 2838.01it/s]


In [9]:
from gensim.models.word2vec import Word2Vec
%time model_tweets = Word2Vec(sentences, workers=4, size=400, min_count=3, window=5, iter=25)

CPU times: user 19.9 s, sys: 171 ms, total: 20.1 s
Wall time: 11.4 s


In [10]:
model_tweets.wv.most_similar('france')

[('bags', 0.7946826815605164),
 ('floor', 0.7802314162254333),
 ('spain', 0.7751898765563965),
 ('2015', 0.7584058046340942),
 ('argentina', 0.7437742948532104),
 ('front', 0.7395506501197815),
 ('streets', 0.7381167411804199),
 ('flour', 0.7271727323532104),
 ('iran', 0.7234978675842285),
 ('tower', 0.7223203182220459)]

In [11]:
model_tweets.init_sims()

In [12]:
import numpy as np

In [13]:
def get_text_embedding(text):
    result = []
    for word in word_tokenize(text.lower()):
        if word in model_tweets.wv:
            result.append(model_tweets.wv[word])

    if len(result):
        result = np.sum(result, axis=0)
    else:
        result = np.zeros(400)
    return result

In [14]:
features = [get_text_embedding(text) for text in tqdm(df.tweet)]

100%|██████████| 6420/6420 [00:03<00:00, 2035.72it/s]


In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(features, df.label, test_size=0.33)

In [17]:
model = LogisticRegression()
model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [18]:
from sklearn.metrics import classification_report

In [19]:
predicted = model.predict(X_test)

In [20]:
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.93      0.92      0.92       969
        real       0.94      0.94      0.94      1150

    accuracy                           0.93      2119
   macro avg       0.93      0.93      0.93      2119
weighted avg       0.93      0.93      0.93      2119



Вариант 2 с использованием *DecisionTreeClassifier*

In [21]:
from sklearn.tree import DecisionTreeClassifier

In [22]:
model = DecisionTreeClassifier(criterion='entropy', max_depth=None, random_state=42)
model.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=42)

In [23]:
predicted = model.predict(X_test)

In [24]:
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.86      0.89      0.88       969
        real       0.90      0.88      0.89      1150

    accuracy                           0.88      2119
   macro avg       0.88      0.88      0.88      2119
weighted avg       0.88      0.88      0.88      2119



Вариант 3 с использованием *SVC*

In [25]:
from sklearn.svm import SVC

In [26]:
model = SVC(kernel='rbf', C=1.0, random_state=42)
model.fit(X_train, y_train)

SVC(random_state=42)

In [27]:
predicted = model.predict(X_test)

In [28]:
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.94      0.94      0.94       969
        real       0.95      0.95      0.95      1150

    accuracy                           0.94      2119
   macro avg       0.94      0.94      0.94      2119
weighted avg       0.94      0.94      0.94      2119



Вариант 4 с использованием *RandomForestClassifier*

In [29]:
from sklearn.ensemble import RandomForestClassifier

In [30]:
model = RandomForestClassifier(n_estimators=1000, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=1000, random_state=42)

In [31]:
predicted = model.predict(X_test)

In [32]:
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.95      0.93      0.94       969
        real       0.95      0.96      0.95      1150

    accuracy                           0.95      2119
   macro avg       0.95      0.95      0.95      2119
weighted avg       0.95      0.95      0.95      2119



Вариант 5 с использованием KNeighborsClassifier

In [33]:
from sklearn.neighbors import KNeighborsClassifier

In [40]:
model = KNeighborsClassifier(n_neighbors=5, p = 2)
model.fit(X_train, y_train)

KNeighborsClassifier()

In [41]:
predicted = model.predict(X_test)

In [42]:
print(classification_report(y_test, predicted))


              precision    recall  f1-score   support

        fake       0.92      0.95      0.93       969
        real       0.95      0.93      0.94      1150

    accuracy                           0.94      2119
   macro avg       0.94      0.94      0.94      2119
weighted avg       0.94      0.94      0.94      2119



###  Что будет, если использовать самый наивный метод?

In [33]:
from sklearn.feature_extraction.text import CountVectorizer

In [34]:
vec = CountVectorizer()

In [35]:
bow = vec.fit_transform(df.tweet)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(features, df.label, test_size=0.33)
model = LogisticRegression()
model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [37]:
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.93      0.92      0.92       996
        real       0.93      0.94      0.93      1123

    accuracy                           0.93      2119
   macro avg       0.93      0.93      0.93      2119
weighted avg       0.93      0.93      0.93      2119



Конечно, мы всегда можем поиграться с предобработкой.

### PyTorch + LSTM

In [37]:
labels = (df.label == 'real').astype(int).to_list()

Нужно заранее задать размер для макксимальной длины предложений.

In [38]:
token_lists = [word_tokenize(text.lower()) for text in df.tweet]
max_len = len(max(token_lists, key=len))

In [39]:
max_len

1592

Это слишком много. Но какая длина обычно?

In [40]:
from collections import Counter
fd = Counter([len(tokens) for tokens in token_lists])

In [41]:
fd.most_common(10)

[(20, 179),
 (25, 174),
 (22, 170),
 (18, 170),
 (19, 167),
 (21, 167),
 (16, 162),
 (15, 161),
 (17, 161),
 (23, 157)]

Зададим максимум 400.

Возьмём те же w2v эмбеддинги.

In [42]:
def get_word_embedding(tokens, max_len):
    result = []
    for i in range(max_len):
        if i < len(tokens):
            word = tokens[i]
            if word in model_tweets.wv:
                result.append(model_tweets.wv[word])
            else:
                result.append(np.zeros(400))
        else:
            result.append(np.zeros(400))
    return result

In [43]:
features = [get_word_embedding(text, 400) for text in tqdm(token_lists)]

100%|██████████| 6420/6420 [00:08<00:00, 729.70it/s]


In [44]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33)

In [45]:
import torch
import torch.nn as nn
import torch.optim as optim

In [46]:
len(features[0][0])

400

In [47]:
len(X_train)

4301

In [48]:
len(X_train[0])

400

In [49]:
len(X_train[0][0])

400

In [50]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(400, 100)
        self.out = nn.Linear(100, 1)

    def forward(self, x):
        embeddings, (shortterm, longterm) = self.lstm(x.transpose(0, 1))
        prediction = torch.sigmoid(self.out(longterm))
        return prediction


net = Net()
print(net)

Net(
  (lstm): LSTM(400, 100)
  (out): Linear(in_features=100, out_features=1, bias=True)
)


In [ ]:
in_data = torch.tensor(X_train).float()
targets = torch.tensor(y_train).float()

In [ ]:
in_data.shape

In [ ]:
optimizer = optim.SGD(net.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [ ]:
def train_one_epoch(in_data, targets, batch_size=16):
    for i in tqdm(range(0, in_data.shape[0], batch_size)):
        batch_x = in_data[i:i + batch_size]
        batch_y = targets[i:i + batch_size]
        optimizer.zero_grad()
        output = net(batch_x)
        loss = criterion(output.reshape(-1), batch_y)
        loss.backward()
        optimizer.step()
    print(loss)

In [ ]:
train_one_epoch(in_data, targets)

Что получилось?

In [ ]:
in_data_test = torch.tensor(X_test).float()
targets_test = torch.tensor(y_test).float()

In [ ]:
with torch.no_grad():
    output = net(in_data_test).reshape(-1)

In [ ]:
result = (output > 0.5) == targets_test

In [ ]:
result.sum().item() / len(result)

Но такую модель надо учить дольше(